In [ ]:
import numpy as np


def Smith_Waterman_algorithm(seq1, seq2, match_score, mismatch_score, gap_score):
    n = len(seq1)
    m = len(seq2)
    S = np.zeros((n + 1, m + 1))
    for i in range(1, n+1):
        for j in range(1, m+1):
            S[i][0] = max(gap_score*j,0)
            S[0][j] = max(gap_score*j,0)
    
    max_element = -1*float('inf')
    max_index = (-1, -1)
    for i in range(1, n + 1):
        for j in range(1, m + 1):
            S[i][j] = max(
                0,
                S[i-1][j-1] + (match_score * (seq1[i-1] == seq2[j-1])) + (mismatch_score * (seq1[i-1] != seq2[j-1])),
                S[i][j-1] + gap_score,
                S[i-1][j] + gap_score
            )
            
            if S[i][j] > max_element:
                max_element = S[i][j]
                max_index = (i, j)


    i = max_index[0]
    j = max_index[1]
    s1 = seq1[:i-1:-1].lower()
    t1 = seq2[:j-1:-1].lower()
    while (i > 0)and(j > 0):
        if abs(S[i][j]) < 1e-10:
            break
        elif abs(S[i-1][j-1] + (match_score * (seq1[i-1] == seq2[j-1])) + (mismatch_score * (seq1[i-1] != seq2[j-1])) - S[i][j]) < 1e-10:
            s1 += seq1[i-1]
            t1 += seq2[j-1]
            i -= 1
            j -= 1
        elif abs(S[i][j-1] + gap_score - S[i][j]) < 1e-10:
            s1 += '_'
            t1 += seq2[j-1]
            j -= 1
        elif abs(S[i-1][j] + gap_score - S[i][j]) < 1e-10:
            s1 += seq1[i-1]
            t1 += '_'
            i -= 1
    
    while (i > 0)and(S[i][j] > 0):
        s1 += seq1[i-1]
        t1 += '_'
        i -= 1
    while (j > 0)and(S[i][j] > 0):
        s1 += '_'
        t1 += seq2[j-1]
        j -= 1
    s1 += seq1[i-1::-1].lower() 
    t1 += seq2[j-1::-1].lower() 
    score =  int(max_element)  
    return score, s1[::-1], t1[::-1]

def main():
    seq1, seq2, match_score, mismatch_score, gap_score= input().split()
    match_score = int(match_score)
    mismatch_score = int(mismatch_score)
    gap_score = int(gap_score)
    print('{} {} {}'.format(*Smith_Waterman_algorithm(seq1, seq2, match_score, mismatch_score, gap_score)))

if __name__ == '__main__':
    main()

In [ ]:
import numpy as np

def three_leveled_Manhattan_Grid_algorithm(seq1, seq2, match_score, mismatch_score, p, sigma):
    Infinity = float('inf')
    lower_level = np.zeros((len(seq1) + 1, len(seq2) + 1)) 
    main_level = np.zeros((len(seq1) + 1, len(seq2) + 1)) 
    upper_level = np.zeros((len(seq1) + 1, len(seq2) + 1))
    
    for i in range(1, len(seq1)+1):
        main_level[i][0] = -Infinity
        lower_level[i][0] = -Infinity
        upper_level[i][0] = p + i * sigma

    for i in range(1, len(seq2)+1):
        main_level[0][i] = -Infinity
        lower_level[0][i] = p + i * sigma
        upper_level[0][i] = -Infinity
   
    
    for i in range(1, len(seq1) + 1):
        for j in range(1, len(seq2) + 1):
            main_level[i][j] = max(main_level[i-1][j-1] + match_score * (seq1[i-1] == seq2[j-1]) + (mismatch_score * (seq1[i-1] != seq2[j-1])),
                    lower_level[i-1][j-1],
                    upper_level[i-1][j-1]
            )

            lower_level[i][j] = max(
                    main_level[i][j-1] + p + sigma,
                    lower_level[i][j-1] + sigma,
        
            )

            upper_level[i][j] = max(
                    main_level[i-1][j] + p + sigma,
                    upper_level[i-1][j] + sigma
            )
    s1 = ''
    t1 = ''
    i = len(seq1)
    j = len(seq2)
    while (i > 0)and(j > 0):
        if abs(main_level[i-1][j-1] + match_score * (seq1[i-1] == seq2[j-1]) + (mismatch_score * (seq1[i-1] != seq2[j-1])) - main_level[i][j]) < 1e-10:
            s1 += seq1[i-1]
            t1 += seq2[j-1]
            i -= 1
            j -= 1
        elif (abs(upper_level[i][j-1] + sigma - main_level[i][j]) < 1e-10) \
        or(abs(main_level[i][j-1]+ p+sigma - main_level[i][j]) < 1e-10):
            s1 += '_'
            t1 += seq2[j-1]
            j -= 1
        elif (abs(lower_level[i-1][j] + sigma - main_level[i][j]) < 1e-10) \
        or(abs(main_level[i-1][j] + p+sigma - main_level[i][j]) < 1e-10):
            s1 += seq1[i-1]
            t1 += '_'
            i -= 1

    while (i > 0):
        s1 += seq1[i-1]
        t1 += '_'
        i -= 1
    while (j > 0):
        s1 += '_'
        t1 += seq2[j-1]
        j -= 1
    score = max(main_level[len(seq1)][len(seq2)], lower_level[len(seq1)][len(seq2)], upper_level[len(seq1)][len(seq2)])    
    return score, s1[::-1], t1[::-1]

def main():
    seq1, seq2, match_score, mismatch_score, p, sigma= input().split()
    match_score = int(match_score)
    mismatch_score = int(mismatch_score)
    p = int(p)
    sigma = int(sigma)
    
    print('{} {} {}'.format(*three_leveled_Manhattan_Grid_algorithm(seq1, seq2, match_score, mismatch_score, p, sigma)))

if __name__ == '__main__':
    main()

In [ ]:
import numpy as np

def three_leveled_Manhattan_Grid_algorithm(seq1, seq2, match_score, mismatch_score, p, sigma):
    Infinity = float('inf')
    lower_level = np.zeros((len(seq1) + 1, len(seq2) + 1)) 
    main_level = np.zeros((len(seq1) + 1, len(seq2) + 1)) 
    upper_level = np.zeros((len(seq1) + 1, len(seq2) + 1))
    
    for i in range(1, len(seq1)+1):
        main_level[i][0] = -Infinity
        lower_level[i][0] = -Infinity
        upper_level[i][0] = p + i * sigma

    for i in range(1, len(seq2)+1):
        main_level[0][i] = -Infinity
        lower_level[0][i] = p + i * sigma
        upper_level[0][i] = -Infinity
   
    
    for i in range(1, len(seq1) + 1):
        for j in range(1, len(seq2) + 1):
            main_level[i][j] = max(main_level[i-1][j-1] + match_score * (seq1[i-1] == seq2[j-1]) + (mismatch_score * (seq1[i-1] != seq2[j-1])),
                    lower_level[i-1][j-1] + match_score * (seq1[i-1] == seq2[j-1]) + (mismatch_score * (seq1[i-1] != seq2[j-1])),
                    upper_level[i-1][j-1] + match_score * (seq1[i-1] == seq2[j-1]) + (mismatch_score * (seq1[i-1] != seq2[j-1]))
                                   
            )

            lower_level[i][j] = max(
                    main_level[i][j-1] + p + sigma,
                    lower_level[i][j-1] + sigma,
                    upper_level[i][j-1] + p + sigma
            )

            upper_level[i][j] = max(
                    main_level[i-1][j] + p + sigma,
                    upper_level[i-1][j] + sigma,
                    lower_level[i-1][j] + p + sigma
                
            )
    s1 = ''
    t1 = ''
    i = len(seq1)
    j = len(seq2)
    while (i > 0)and(j > 0):
        if (i>0 and j>0 and (main_level[i-1][j-1] + match_score * (seq1[i-1] == seq2[j-1]) + (mismatch_score * (seq1[i-1] != seq2[j-1])))== main_level[i][j]):
            s1 += seq1[i-1]
            t1 += seq2[j-1]
            i -= 1
            j -= 1
        elif (i>0 and upper_level[i][j-1] == main_level[i][j]):
            s1 += '_'
            t1 += seq2[j-1]
            j -= 1
        elif (lower_level[i-1][j] == main_level[i][j]):
            s1 += seq1[i-1]
            t1 += '_'
            i -= 1
    s1 = ' '.join([seq1[j] for j in range(-1, -(len(seq1)+1),-1)])
    t1 = ' '.join([seq2[j] for j in range(-1, -(len(seq2)+1),-1)])
    score = max(main_level[len(seq1)][len(seq2)], lower_level[len(seq1)][len(seq2)], upper_level[len(seq1)][len(seq2)])    
    return score, s1[::-1], t1[::-1]

def main():
    seq1, seq2, match_score, mismatch_score, p, sigma= input().split()
    match_score = int(match_score)
    mismatch_score = int(mismatch_score)
    p = int(p)
    sigma = int(sigma)
    
    print('{} {} {}'.format(*three_leveled_Manhattan_Grid_algorithm(seq1, seq2, match_score, mismatch_score, p, sigma)))

if __name__ == '__main__':
    main()